In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from itertools import groupby
from operator import itemgetter

/Users/yelenanevel/opt/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
sephora = pd.read_csv('../data/sephora_intial_clean.csv')

In [3]:
credo = pd.read_csv('../data/credo_initial_clean.csv')

In [4]:
follain = pd.read_csv('../data/follain_initial_clean.csv')

In [5]:
sephora.head()

,name,brand,category,price,ingredients,no_reviews,hearts,size1,size2,url,final_size,store,type
0,Protini Polypeptide Moisturizer,Drunk Elephant,moisturizing-cream-oils-mists,68.0,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",3000,216935,1.69,0.0,https://www.sephora.com/product/protini-tm-pol...,1.69,Sephora,clean
1,The Water Cream,Tatcha,moisturizing-cream-oils-mists,68.0,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",2000,197492,0.00,1.7,https://www.sephora.com/product/the-water-crea...,1.70,Sephora,clean
2,Ultra Facial Cream,Kiehl's Since 1851,moisturizing-cream-oils-mists,32.0,"Water, Glycerin, Cyclohexasiloxane, Squalane, ...",943,87617,0.00,1.7,https://www.sephora.com/product/ultra-facial-c...,1.70,Sephora,NaN
3,CC+ Cream with SPF 50+,IT Cosmetics,moisturizing-cream-oils-mists,39.5,"Titanium Dioxide 90%, Zinc Oxide 63%",3000,225410,1.08,0.0,https://www.sephora.com/product/your-skin-but-...,1.08,Sephora,NaN
4,The Dewy Skin Cream,Tatcha,moisturizing-cream-oils-mists,68.0,"Water, Saccharomyces, Glycerin, Propanediol, D...",1000,85005,0.00,1.7,https://www.sephora.com/product/the-dewy-skin-...,1.70,Sephora,clean


In [6]:
credo.head()

,name,brand,category,price,ingredients,url,type,store
0,Vitamin B Enzyme Cleansing Oil + Makeup Remover,One Love Organics,all-cleansers,42.0,"helianthus annuus seed oil, caprylic, di, poly...",https://credobeauty.com//collections/all-clean...,clean,Credo
1,Exfoliating Instant Facial,Goop,all-cleansers,125.0,"aloe barbadensis leaf juice, vitis vinifera ju...",https://credobeauty.com//collections/all-clean...,clean,Credo
2,Hydrating Accelerator,Josh Rosebrook,all-cleansers,35.0,"aloe vera leaf juice, jojoba oil, marula seed ...",https://credobeauty.com//collections/all-clean...,clean,Credo
3,Co Q10 Toner,Indie Lee,all-cleansers,34.0,"purified water, aloe barbadensis, coenzyme q10...",https://credobeauty.com//collections/all-clean...,clean,Credo
4,Brightening Cleanser,Indie Lee,all-cleansers,34.0,"purified water, decyl glucoside, disodium coco...",https://credobeauty.com//collections/all-clean...,clean,Credo


In [7]:
follain.head()

,name,brand,category,price,ingredients,store,url,type
0,Brightening Cleanser,Indie Lee,cleansers,34.0,"Water, Decyl Glucoside, Disodium Coco Glucosid...",Follain,https://follain.com//collections/skincare-clea...,clean
1,Ocean Cleanser,OSEA,cleansers,48.0,"Water, Algae Extract, Decyl Glucoside, Sodium ...",Follain,https://follain.com//collections/skincare-clea...,clean
2,Regenerating Cleanser,Tata Harper,cleansers,84.0,"Aloe Vera Leaf Juice, Cetearyl Alcohol, Cetear...",Follain,https://follain.com//collections/skincare-clea...,clean
3,Vitamin B Cleansing Oil & Makeup Remover,One Love Organics,cleansers,42.0,"Helianthus Annus Seed Oil, Caprylic, Di-PPG-2 ...",Follain,https://follain.com//collections/skincare-clea...,clean
4,Fantastic Face Wash,Ursa Major,cleansers,28.0,"Aloe Barbadensis Leaf Juice, Lauryl Glucoside,...",Follain,https://follain.com//collections/skincare-clea...,clean


#### Sephora

In [8]:
sephora = sephora[['name', 'brand', 'category', 'price', 'ingredients', 'store', 'url', 'type']]
sephora['website'] = 'Sephora.com'

In [9]:
sephora.category.unique()

array(['moisturizing-cream-oils-mists', 'cleanser', 'facial-treatments',
       'eye-treatment-dark-circle-treatment', 'facial-treatment-masks',
       'sunscreen-sun-protection', 'lip-treatments'], dtype=object)

In [10]:
sephora['category'] = sephora['category'].str.replace('moisturizing-cream-oils-mists', "moisturizer")
sephora['category'] = sephora['category'].str.replace('facial-treatments', "treatment")
sephora['category'] = sephora['category'].str.replace('eye-treatment-dark-circle-treatment', "eye")
sephora['category'] = sephora['category'].str.replace('facial-treatment-masks', "mask")
sephora['category'] = sephora['category'].str.replace('sunscreen-sun-protection', "sun")
sephora['category'] = sephora['category'].str.replace('lip-treatments', "lip")

#### Credo

In [11]:
credo.head()

,name,brand,category,price,ingredients,url,type,store
0,Vitamin B Enzyme Cleansing Oil + Makeup Remover,One Love Organics,all-cleansers,42.0,"helianthus annuus seed oil, caprylic, di, poly...",https://credobeauty.com//collections/all-clean...,clean,Credo
1,Exfoliating Instant Facial,Goop,all-cleansers,125.0,"aloe barbadensis leaf juice, vitis vinifera ju...",https://credobeauty.com//collections/all-clean...,clean,Credo
2,Hydrating Accelerator,Josh Rosebrook,all-cleansers,35.0,"aloe vera leaf juice, jojoba oil, marula seed ...",https://credobeauty.com//collections/all-clean...,clean,Credo
3,Co Q10 Toner,Indie Lee,all-cleansers,34.0,"purified water, aloe barbadensis, coenzyme q10...",https://credobeauty.com//collections/all-clean...,clean,Credo
4,Brightening Cleanser,Indie Lee,all-cleansers,34.0,"purified water, decyl glucoside, disodium coco...",https://credobeauty.com//collections/all-clean...,clean,Credo


In [12]:
credo['website'] = 'CredoBeauty.com'

In [13]:
credo.category.unique()

array(['all-cleansers', 'all-moisturizers', 'masks-1', 'treatments',
       'eye-care', 'sunscreen', 'lip-care'], dtype=object)

In [14]:
credo['category'] = credo['category'].str.replace('all-cleansers', "cleanser")
credo['category'] = credo['category'].str.replace('all-moisturizers', "moisturizer")
credo['category'] = credo['category'].str.replace('masks-1', "mask")
credo['category'] = credo['category'].str.replace('eye-care', "eye")
credo['category'] = credo['category'].str.replace('treatments', "treatment")
credo['category'] = credo['category'].str.replace('sunscreen', "sun")
credo['category'] = credo['category'].str.replace('lip-care', "lip")

#### Follain

In [15]:
follain.head()

,name,brand,category,price,ingredients,store,url,type
0,Brightening Cleanser,Indie Lee,cleansers,34.0,"Water, Decyl Glucoside, Disodium Coco Glucosid...",Follain,https://follain.com//collections/skincare-clea...,clean
1,Ocean Cleanser,OSEA,cleansers,48.0,"Water, Algae Extract, Decyl Glucoside, Sodium ...",Follain,https://follain.com//collections/skincare-clea...,clean
2,Regenerating Cleanser,Tata Harper,cleansers,84.0,"Aloe Vera Leaf Juice, Cetearyl Alcohol, Cetear...",Follain,https://follain.com//collections/skincare-clea...,clean
3,Vitamin B Cleansing Oil & Makeup Remover,One Love Organics,cleansers,42.0,"Helianthus Annus Seed Oil, Caprylic, Di-PPG-2 ...",Follain,https://follain.com//collections/skincare-clea...,clean
4,Fantastic Face Wash,Ursa Major,cleansers,28.0,"Aloe Barbadensis Leaf Juice, Lauryl Glucoside,...",Follain,https://follain.com//collections/skincare-clea...,clean


In [16]:
follain['website'] = 'Follain.com'

In [17]:
follain.category.unique()

array(['cleansers', 'toners', 'moisturizers', 'treatments-and-serums',
       'masks-exfoliants', 'eye-care', 'lip-care', 'sun-care'],
      dtype=object)

In [18]:
follain['category'] = follain['category'].str.replace('cleansers', "cleanser")
follain['category'] = follain['category'].str.replace('moisturizers', "moisturizer")
follain['category'] = follain['category'].str.replace('masks-exfoliants', "mask")
follain['category'] = follain['category'].str.replace('eye-care', "eye")
follain['category'] = follain['category'].str.replace('treatments-and-serums', "treatment")
follain['category'] = follain['category'].str.replace('sun-care', "sun")
follain['category'] = follain['category'].str.replace('lip-care', "lip")
follain['category'] = follain['category'].str.replace('toners', "cleanser")

#### Combining DataFrames

In [19]:
products = pd.concat([sephora, credo, follain], axis = 0, ignore_index = True, sort = False)
products.head()

,name,brand,category,price,ingredients,store,url,type,website
0,Protini Polypeptide Moisturizer,Drunk Elephant,moisturizer,68.0,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",Sephora,https://www.sephora.com/product/protini-tm-pol...,clean,Sephora.com
1,The Water Cream,Tatcha,moisturizer,68.0,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",Sephora,https://www.sephora.com/product/the-water-crea...,clean,Sephora.com
2,Ultra Facial Cream,Kiehl's Since 1851,moisturizer,32.0,"Water, Glycerin, Cyclohexasiloxane, Squalane, ...",Sephora,https://www.sephora.com/product/ultra-facial-c...,NaN,Sephora.com
3,CC+ Cream with SPF 50+,IT Cosmetics,moisturizer,39.5,"Titanium Dioxide 90%, Zinc Oxide 63%",Sephora,https://www.sephora.com/product/your-skin-but-...,NaN,Sephora.com
4,The Dewy Skin Cream,Tatcha,moisturizer,68.0,"Water, Saccharomyces, Glycerin, Propanediol, D...",Sephora,https://www.sephora.com/product/the-dewy-skin-...,clean,Sephora.com


In [20]:
products.tail()

,name,brand,category,price,ingredients,store,url,type,website
3133,Mineral Sunscreen Unscented SPF 30,Suntegrity,sun,24.0,"Zinc, Aloe Vera Leaf Juice, Camellia Sinensis,...",Follain,https://follain.com//collections/skincare-sun-...,clean,Follain.com
3134,Super Shield Sport Stick Sunscreen SPF 50,Babo Botanicals,sun,16.0,"Zinc Oxide, Beeswax, Bisabolol, Shea Butter, T...",Follain,https://follain.com//collections/skincare-sun-...,clean,Follain.com
3135,Mineral Sunscreen SPF 30 Fragrance-Free,Love Sun Body,sun,36.0,"Zinc Oxide 20.4%, Almond Oil, Water, Benzoic A...",Follain,https://follain.com//collections/skincare-sun-...,clean,Follain.com
3136,Mineral Sunscreen SPF 30 Lightly Scented,Love Sun Body,sun,36.0,"Glycerin, Cetyl Alcohol, Zinc Oxide 20.4%, Ing...",Follain,https://follain.com//collections/skincare-sun-...,clean,Follain.com
3137,Mineral Sunscreen SPF 30,Suntegrity,sun,36.0,"Zinc, Aloe Vera Leaf Juice, Camellia Sinensis,...",Follain,https://follain.com//collections/skincare-sun-...,clean,Follain.com


In [21]:
#these are all tools and rollers and not products, I will drop them
products[products.ingredients.isnull()]

,name,brand,category,price,ingredients,store,url,type,website
3069,The Gua Sha Facial Lifting Tool,Mount Lai,treatment,28.0,NaN,Follain,https://follain.com//collections/skincare-trea...,clean,Follain.com
3073,The Rose Quartz Facial Roller,Mount Lai,treatment,38.0,NaN,Follain,https://follain.com//collections/skincare-trea...,clean,Follain.com
3076,The Mini Jade Facial Roller,Mount Lai,treatment,20.0,NaN,Follain,https://follain.com//collections/skincare-trea...,clean,Follain.com
3088,Dual-Action Jade Facial Roller,Province Apothecary,treatment,44.0,NaN,Follain,https://follain.com//collections/skincare-trea...,clean,Follain.com


In [22]:
#dropping any products that ingredients were unable to be gathered. these are rollers and tools and products
products = products[products.ingredients.notnull()]

In [23]:
#convert price to str and add $, so it prints out in a nice format to the screen
products.price = products.price.astype(str)
products.price = '$' + products.price

In [24]:
products.head()

,name,brand,category,price,ingredients,store,url,type,website
0,Protini Polypeptide Moisturizer,Drunk Elephant,moisturizer,$68.0,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",Sephora,https://www.sephora.com/product/protini-tm-pol...,clean,Sephora.com
1,The Water Cream,Tatcha,moisturizer,$68.0,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",Sephora,https://www.sephora.com/product/the-water-crea...,clean,Sephora.com
2,Ultra Facial Cream,Kiehl's Since 1851,moisturizer,$32.0,"Water, Glycerin, Cyclohexasiloxane, Squalane, ...",Sephora,https://www.sephora.com/product/ultra-facial-c...,NaN,Sephora.com
3,CC+ Cream with SPF 50+,IT Cosmetics,moisturizer,$39.5,"Titanium Dioxide 90%, Zinc Oxide 63%",Sephora,https://www.sephora.com/product/your-skin-but-...,NaN,Sephora.com
4,The Dewy Skin Cream,Tatcha,moisturizer,$68.0,"Water, Saccharomyces, Glycerin, Propanediol, D...",Sephora,https://www.sephora.com/product/the-dewy-skin-...,clean,Sephora.com


In [25]:
products['ingredients'] = products['ingredients'].str.replace('&', "")
products['ingredients'] = products['ingredients'].str.replace('^', "")
products['ingredients'] = products['ingredients'].str.replace('+', "")
products['ingredients'] = products['ingredients'].str.replace('100% fractionally distilled ', "")
products['ingredients'] = products['ingredients'].str.replace('100% ', "")
products['ingredients'] = products['ingredients'].str.replace('a ', "")
products['ingredients'] = products['ingredients'].str.replace('and ', "")
products['ingredients'] = products['ingredients'].str.replace('\u2028', "")
products['ingredients'] = products['ingredients'].str.replace(']', "")
products['ingredients'] = products['ingredients'].str.replace('[', "")

for i in products.index:
    list_of_ingredients = []
    split = products.ingredients[i].split(", ")
    for j in range(len(split)):
        results = split[j].split(":")[0]
        results = results.strip() #stripping any white space 
        list_of_ingredients.append(results)
    products.loc[i, 'ingredients'] = ', '.join(list_of_ingredients)

In [26]:
# all_ingredients = []
# #this code goes through all of the rows of ingredients and combines them in to one list
# for i in products.index:
#     list_ingredients = products.ingredients[i].split(', ') #splits a row ingredients into a list 
#     for j in list_ingredients:     
#         all_ingredients.append(j.strip().lower())
# sorted(set(all_ingredients))[3000:4000]

In [27]:
products.website = '<a href="' + products['url'] + '">' + products['website'] + '</a>'

#### Fuzzy String Matching

In [28]:
all_ingredients = []
#this code goes through all of the rows of ingredients and combines them in to one list
for i in products.index:
    list_ingredients = products.ingredients[i].split(', ') #splits a row ingredients into a list 
    for j in list_ingredients:     
        all_ingredients.append(j.strip().lower()) 

ingredients = pd.DataFrame([(x,all_ingredients.count(x)) for x in set(all_ingredients)],
                           columns = ['ingredient', 'count']).sort_values(by='ingredient')

ingredients = ingredients[ingredients.ingredient != '']
ingredients = ingredients[ingredients.ingredient != '1']
ingredients = ingredients[ingredients.ingredient != '3']
ingredients = ingredients[ingredients.ingredient != '6']
ingredients = ingredients[ingredients.ingredient != '3%']
ingredients = ingredients[ingredients.ingredient != '1,2']

In [29]:
#https://stackoverflow.com/questions/17366788/python-split-list-based-on-first-character-of-word
letters = []
for letter, words in groupby(sorted(ingredients.ingredient), key=itemgetter(0)):
    for word in words:
        letters.append(letter)
letters

#first letter of the ingredient name
ingredients['letters'] = letters
ingredients.head()

,ingredient,count,letters
5463,1 compostable natural rayon wipe contains 2,1,1
6437,10hydroxydecanoic acid,7,1
4888,110decanediol,1,1
6462,12hexanediol,318,1
1864,15 alkyl benzoate,2,1


In [30]:
#empty list that the rations will go in to
fuzzy = []
#looping through the letters of the alphabet
for i in sorted(set(ingredients['letters'])):
    #when ingredients starts with that letter
    df = ingredients[ingredients['letters'] == i]
    #calculating ratios 
    for k in df.ingredient:
        for j in df.ingredient:
            fuzzy.append((k, j, fuzz.ratio(k, j), df[df.ingredient == k]['count'].values[0],
                          df[df.ingredient == j]['count'].values[0]))
#creating a dataframe of ratios
fuzzy_ratio = pd.DataFrame(fuzzy, columns = ['ingredient1', 'ingredients2', 'ratio', 'word1 count', 'word2 count'])
fuzzy_ratio

,ingredient1,ingredients2,ratio,word1 count,word2 count
0,1 compostable natural rayon wipe contains 2,1 compostable natural rayon wipe contains 2,100,1,1
1,1 compostable natural rayon wipe contains 2,10hydroxydecanoic acid,12,1,7
2,1 compostable natural rayon wipe contains 2,110decanediol,14,1,1
3,1 compostable natural rayon wipe contains 2,12hexanediol,11,1,318
4,1 compostable natural rayon wipe contains 2,15 alkyl benzoate,20,1,2
...,...,...,...,...,...
3483726,|natural constituents of essential oils,| limonene,29,1,1
3483727,|natural constituents of essential oils,| linalool,29,1,1
3483728,|natural constituents of essential oils,|natural constituents of essential oils,100,1,1
3483729,∙ organically grown,∙ organically grown,100,1,1


In [31]:
#only looking at top 80 ratio
fuzzy_ratio = fuzzy_ratio[fuzzy_ratio.ratio != 100]
top75=fuzzy_ratio[fuzzy_ratio.ratio > 75]
top75.head()

,ingredient1,ingredients2,ratio,word1 count,word2 count
77,30ethyl ascorbic acid,3oethyl ascorbic acid,95,1,25
102,3oethyl ascorbic acid,30ethyl ascorbic acid,95,25,1
114,4t butylcyclohexanol,4tbutylcyclohexanol,97,2,4
117,4tbutylcyclohexanol,4t butylcyclohexanol,97,4,2
3246,acacidealbatflower,acacidealbatflower cera,88,1,4


In [32]:
#checking for duplicates if the order of column is just reversed

#https://stackoverflow.com/questions/24676705/pandas-drop-duplicates-if-reverse-is-present-between-two-columns
top75['check_string'] = top75.apply(lambda row: ''.join(sorted([row['ingredient1'], row['ingredients2']])), axis=1)

#https://stackoverflow.com/questions/12497402/python-pandas-remove-duplicates-by-columns-a-keeping-the-row-with-the-highest
top75_nodupes = top75.groupby('check_string', group_keys=False).apply(lambda x: x.loc[x['word1 count'].idxmax()])
top75_nodupes

/Users/yelenanevel/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,ingredient1,ingredients2,ratio,word1 count,word2 count,check_string
check_string,,,,,,
30ethyl ascorbic acid3oethyl ascorbic acid,3oethyl ascorbic acid,30ethyl ascorbic acid,95,25,1,30ethyl ascorbic acid3oethyl ascorbic acid
4t butylcyclohexanol4tbutylcyclohexanol,4tbutylcyclohexanol,4t butylcyclohexanol,97,4,2,4t butylcyclohexanol4tbutylcyclohexanol
acacidealbatfloweracacidealbatflower cera,acacidealbatflower cera,acacidealbatflower,88,4,1,acacidealbatfloweracacidealbatflower cera
acacidecurrens flower ceraacacidecurrens flower wax,acacidecurrens flower wax,acacidecurrens flower cera,90,18,1,acacidecurrens flower ceraacacidecurrens flowe...
acacisenegalacacisenegal gum,acacisenegal gum,acacisenegal,86,50,7,acacisenegalacacisenegal gum
...,...,...,...,...,...,...
ziziphus jujubfruit extractzizyphus jujubfruit extract,ziziphus jujubfruit extract,zizyphus jujubfruit extract,96,10,3,ziziphus jujubfruit extractzizyphus jujubfruit...
ziziphus jujubfruit extractzizyphus jujubseed extract,ziziphus jujubfruit extract,zizyphus jujubseed extract,79,10,4,ziziphus jujubfruit extractzizyphus jujubseed ...
zizyphus jujubextractzizyphus jujubfruit extract,zizyphus jujubfruit extract,zizyphus jujubextract,88,3,2,zizyphus jujubextractzizyphus jujubfruit extract


In [33]:
#pivot table represents all the ingredients to keep vs the ones to change
pivot = top75_nodupes.pivot(index = 'ingredient1', columns = 'ingredients2', values = 'word1 count')
pivot

ingredients2,30ethyl ascorbic acid,4t butylcyclohexanol,acacidealbatflower,acacidecurrens flower cera,acacisenegal,acacivictoriae fruit extract,acai palm extract,acanthopanax senticosus extract,acer rubrum extract,acer saccarum extract,...,zingiber officinale root e x tr act,zingiber officinale root extract,zingiber officinale root oil,zingiber officinale water,zingiber officinalis root oil,zingiber zerumbet extract,zingibert officinalis extract,zizyphus jujubextract,zizyphus jujubfruit extract,zizyphus jujubseed extract
ingredient1,,,,,,,,,,,,,,,,,,,,,
3oethyl ascorbic acid,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4tbutylcyclohexanol,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
acacidealbatflower cera,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
acacidecurrens flower wax,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
acacisenegal gum,NaN,NaN,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zingiber officinale water,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
zingiber officinalis root oil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
ziziphus jujubfruit extract,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0,10.0


In [34]:
#based on the pivot table create a dataframe with columns to replace and value to replace with
ingredient_replace = []
for i in pivot.columns:
    #choosing the maximum number of counts in pivot column to replace with
    ingredient_replace.append((i, pivot[pivot[i] == pivot.loc[:, i].max()].index.values[0]))
ingredient_replace_df = pd.DataFrame(ingredient_replace, columns = ['to_replace', 'replace_with'])
ingredient_replace_df

,to_replace,replace_with
0,30ethyl ascorbic acid,3oethyl ascorbic acid
1,4t butylcyclohexanol,4tbutylcyclohexanol
2,acacidealbatflower,acacidealbatflower cera
3,acacidecurrens flower cera,acacidecurrens flower wax
4,acacisenegal,acacisenegal gum
...,...,...
3472,zingiber zerumbet extract,zingiber aromaticus extract
3473,zingibert officinalis extract,zingiber officinale root extract
3474,zizyphus jujubextract,ziziphus jujubfruit extract
3475,zizyphus jujubfruit extract,ziziphus jujubfruit extract


In [35]:
#need to replace now in the full data set
for i in products.index:
    split_list = products.loc[i, 'ingredients'].split(', ')
    for index, item in enumerate(split_list):
        if item in ingredient_replace_df['to_replace'].values:
            split_list[index] = ingredient_replace_df[ingredient_replace_df['to_replace']==item]['replace_with'].values[0]
    products.loc[i, 'ingredients'] = ', '.join(split_list)


In [36]:
products.to_csv('../data/products_combined.csv', index = False)